**Step 3 Feature Engineering**

In [2]:
import pandas as pd

In [3]:
df=pd.read_csv(r"C:\Users\HomePC\Crypto-BuySell-Classifier-Model\notebooks\data\processed\BTCUSDT_1d.csv")
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   open_time           1000 non-null   int64  
 1   open                1000 non-null   float64
 2   high                1000 non-null   float64
 3   low                 1000 non-null   float64
 4   close               1000 non-null   float64
 5   volume              1000 non-null   float64
 6   close_time          1000 non-null   int64  
 7   quote_asset_volume  1000 non-null   float64
 8   num_trades          1000 non-null   float64
 9   taker_base_volume   1000 non-null   float64
 10  taker_quote_volume  1000 non-null   float64
dtypes: float64(9), int64(2)
memory usage: 86.1 KB


,open_time,open,high,low,close,volume,close_time,quote_asset_volume,num_trades,taker_base_volume,taker_quote_volume
0,1677974400000,22346.57,22662.09,22189.22,22430.24,154841.75786,1678060799999,3.473011e+09,4835978.0,77394.35765,1.735989e+09
1,1678060800000,22430.24,22602.19,22258.00,22410.00,203751.82957,1678147199999,4.569102e+09,6471278.0,102110.26304,2.289889e+09
2,1678147200000,22409.41,22557.91,21927.00,22197.96,292519.80912,1678233599999,6.517595e+09,7813394.0,145498.64219,3.242137e+09
3,1678233600000,22198.56,22287.00,21580.00,21705.44,301460.57272,1678319999999,6.641172e+09,8018963.0,150008.02488,3.304860e+09
4,1678320000000,21704.37,21834.99,20042.72,20362.22,443658.28584,1678406399999,9.403029e+09,10076702.0,217900.59621,4.619804e+09


In [4]:
#1-day return
df['return_1d'] = df['close'].pct_change(1)
df['return_1d']

0           NaN
1     -0.000902
2     -0.009462
3     -0.022188
4     -0.061884
         ...   
995    0.016930
996   -0.010533
997    0.035642
998    0.009393
999   -0.004476
Name: return_1d, Length: 1000, dtype: float64

In [5]:
#7 day return
df['return_7d'] = df['close'].pct_change(7)
df['return_7d']

0           NaN
1           NaN
2           NaN
3           NaN
4           NaN
         ...   
995   -0.042456
996   -0.060142
997   -0.011697
998    0.054211
999    0.068081
Name: return_7d, Length: 1000, dtype: float64